## Exercise 1. Provide a histogram of word repetitions

You are asked to design the map and reduce functions to provide a histogram of word repetitions in one or multiple text documents. That is, the number of words that are repeated once, the number of words that are repeated twice, the number of words that are repeated three times, etc.

**Input**: A document or multiple documents, containing plain text. You can assume that each line (or document) is given a random ID that will be used as input key for the map phase, and the input values are the lines of text or entire documents. So the input may look like this:

    (1, Welcome to the World)
    (2, of Big Big Data)
    (3, Welcome World Bye)

**Output**: Your output should say how many words repeat 1, 2, etc. times, where the number of times is given as output key, and the total number of words repetitions as the value (i.e. `(number of repetitions, number of words)`). Thus, for the above input, the output should be as follows:

    (1 time - 5 words)
    (2 times - 3 words)

**Hint**: You can concatenate as many MapReduce processes as you like, so for example you could first apply the solution for the Word Count and take that as input for the a second stage.

### Solution: 

![MapReduce 1](img/mapreduce1.png)

We are implementing two MapReduce operations to build the histogram of word repetitions. 

### MapReduce 1

- **Input**: Key-value pairs $(k1, v1)$ where $k1$ is the line number and $v1$ is the sentence.

- **Map Function**: Maps $(k1, v1)$ into $(k2, v2)$ where $k2$ is the word and $v2$ is assigned a value of 1. (E.g., ('Welcome', 1)).

- **Shuffle**: Performs shuffle operation to create a list of values $list(v2)$ corresponding to each unique key $k2$.

- **Reduce Function**: Uses addition operation to add the values in the list $list(v2)$ for each key $k3$ in $k2$, assigning it a value $v3$.

- **Output**: $(k3, v3)$ contains the word and the count of its occurrences.


### MapReduce 2

- **Input**: Key-value pairs $(k3, v3)$ where $k3$ is the word and $v3$ is it's occurance count.

- **Map Function**: Maps $(k3, v3)$ into $(k4, v4)$ where $k4$ is $v3$ and $v4$ is assigned a value of 1.

- **Combiners**: A mini-reducer is applied on (k4,v4) with addition to add the values in the local node for each unique key. 

- **Shuffle**: Performs shuffle operation to create a list of values $list(v4)$ corresponding to each unique key $k4$.

- **Reduce Function**: Uses addition operation to add the values in the list $list(v4)$ for each key $k5$ in $k4$, assigning it as value $v5$.

- **Output**: $(k5, v5)$ contains the number of word repetitions and the number of words with corresponding repetitions.


## Exercise 2. Provide a histogram of the length of the words. 


You are asked to design the map and reduce functions to provide a histogram of the length of the words in one or multiple text documents. Word repetition is not a problem, so if you have the word 'bye' twice in your document, you would add 2 to the number of words of length 3.

**Input**: Same as the previous exercise. So the input may look like this:

    (1, Welcome to the World)
    (2, of Big Big Data)
    (3, Welcome World Bye)
    
**Output**: Your output should say how many words have a given length, where the length is given as output key, and the total number of words with such length is provided as the value (i.e. `(length, number of words)`). Thus, for the above input, the output should be as follows:

    (2,2)
    (3,4)
    (4,1)
    (5,2)
    (7,2)



### Solution: 

![MapReduce 2](img/EX-02.png)

We are implementing a MapReduce operation to build the histogram of length of words. 

### MapReduce 

- **Input**: Key-value pairs $(k1, v1)$ where $k1$ is the line number and $v1$ is the sentence.

- **Map Function**: For each $k1$, we split the text $(v1)$ into words then get the length of the word where we will use as the key $(k2)$ for the output. Therefor $k2 = len(v1.split()[i])$ is the ith word of splitted text. For each $k2$ we will set the value $v2$ to $1$ we then obtained a list of key-value $list((k2, v2)) = list((len(v1.split()[i]), 1))$ as an output from the mapping stage. We also used mini reducer $lambda x, y: x+y$ to combine value with the same key from the mapping output.

- **Shuffle**: The output from the mapping stage are sorted according to key $(k2)$ and the value withing the same key will be put together in the array. 

- **Reduce Function**: Again, we apply reduce function $lambda x, y: x+y$ to sum the value in the list for each key. Hence, we obtained (key, value) represent (word length, number of words)

- **Output**: $(k3, v3)$ contains the word and the length of the word.


## Exercise 4. Find a list of common friends *between pairs of friends*.

Obtain a list of common friends for each pair of friends. That is, for each two people that are friends (assuming friendship is bi-directional), you need to find the friends in common.

You are asked to design the map and reduce functions that read a file that contains a list of friends for each person. Using map and reduce functions you should find for each pair of friends, the list of common friends.

**Input**: A file with the following format: Person -> List of friends. 

```
Isaac -> Mikel John Lucy
Mikel -> Isaac John Lucy Claudia
John -> Isaac Mikel Lucy Claudia
Lucy -> Isaac Mikel John Claudia
Claudia -> Mikel John Lucy
```

**Output**: (pair of friends, list of common friends)
```
(Isaac, Mikel)    -> [John, Lucy]
(Claudia, Mikel)  -> [John, Lucy]
(John, Lucy)      -> [Claudia, Isaac, Mikel]
(Isaac, John)     -> [Lucy, Mikel]
(Isaac, Lucy)     -> [John, Mikel]
(John, Mikel)     -> [Claudia, Isaac, Lucy]
(Lucy, Mikel)     -> [Claudia, Isaac, John]
(Claudia, John)   -> [Lucy, Mikel]
(Claudia, Lucy)   -> [John, Mikel]



### Solution: 

![MapReduce 4](img/EX4.png)

We are implementing a MapReduce operation to find list of common friends between pairs of friends.

### MapReduce Process for Finding Common Friends

- **Input**: Key-value pairs (`k1`, `v1`) where `k1` is a person and `v1` is a list of friends associated with that person, formatted as `[v1(1), v1(2), ... v1(N)]`.

- **Map Function**: Outputs key-value pairs (`k2`, `v2`) for each friend `v1(j)` in `v1`, with `k2` being a sorted tuple `(k1, v1(j))` representing a pair of potential friends and `v2` as the set of all friends from `v1` excluding `v1(j)`, effectively `set(v1) - {v1(j)}`.

- **Shuffle**: Organizes the output from the map function by `k2`, aggregating all `v2` sets associated with the same key into a list (`k3`, `v3`), preparing for the reduction phase by grouping common connections for each unique pair of friends.

- **Reduce Function**: Applies an intersection across all sets in `v3` for each `k3`, producing `k4` (identical to `k3`) and `v4`, which contains the final set of mutual friends shared between the two individuals in each pair. The reduce operation is defined as `reduce(lambda x, y: x.intersection(y), v3)`.

- **Output**: The final result is represented as key-value pairs (`k4`, `v4`), where `k4` is each unique pair of friends and `v4` is the set of their common friends.